In [44]:
import numpy as np           # Handle matrices
from vizdoom import *        # Doom Environment

import random                # Handling random number generation
import time                  # Handling time calculation

from collections import deque # Ordered collection with ends
import matplotlib.pyplot as plt # Display graphs

from torchvision import transforms, utils
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import PIL
from collections import namedtuple

## Hyper-parameters

In [37]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
action_size = 3              # 3 possible actions: left, right, shoot
learning_rate =  0.0002      # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 500        # Total episodes for training
max_steps = 100             # Max possible steps in an episode
batch_size = 64             

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Q learning hyperparameters
gamma = 0.95               # Discounting rate

### MEMORY HYPERPARAMETERS
pretrain_length = batch_size   # Number of experiences stored in the Memory when initialized for the first time
memory_size = 1000000          # Number of experiences the Memory can keep

### MODIFY THIS TO FALSE IF YOU JUST WANT TO SEE THE TRAINED AGENT
training = True

## TURN THIS TO TRUE IF YOU WANT TO RENDER THE ENVIRONMENT
episode_render = False

## Create Env

In [38]:
def create_environment():
    game = DoomGame()
    
    # Load the correct configuration
    game.load_config("basic.cfg")
    
    game.init()
    
    # Here our possible actions
    left = [1, 0, 0]
    right = [0, 1, 0]
    shoot = [0, 0, 1]
    possible_actions = [left, right, shoot]
    
    return game, possible_actions

## Data Loader

In [45]:
class ReplayMemory(object):
    """
    Class that handles the replay buffer and takes in the raw numpy and transforms it as necessary
    Transforms:
        Crop out the unnecessary parts of image
        Normalize to 0-1
        Resize to 84x84
    Also handles stacking the frames together
    """

    def __init__(self, device, stack_size=4, memory_size = 1000000):
        self.memory = deque(maxlen=memory_size)
        self.img_transforms = transforms.Compose([transforms.Resize((84,84)), transforms.ToTensor()])
        self.stacked_queue = deque(maxlen=stack_size)
        self.device = device
        self.stack_size = stack_size
        self.MemoryItem = namedtuple('MemoryItem', ['state', 'action', 'reward', 'next_state', 'done'])
        
    def __len__(self):
        return len(self.memory)
    
    def _transform(self, x):
        img = PIL.Image.fromarray(x)
        img_cropped = transforms.functional.crop(img,30,30,70,100)
        return self.img_transforms(img_cropped).to(device)
    
    def _reset_stacked_queue(self, x):
        # reset stacked queue to the same frame
        for _ in range(self.stack_size-1):
            self.stacked_queue.append(x)
            
    def _get_stacked_state(self):
        return torch.cat(list(self.stacked_queue), 0)
            
    def _handle_state(self, state, reset_stack):
        if state is None:
            return self._get_stacked_state()
        else:
            state = self._transform(state)
            if reset_stack:
                self._reset_stacked_queue(state)
            self.stacked_queue.append(state)
            return self._get_stacked_state()
    
    def push(self, state, action, reward, next_state, done, reset_stack=False):
        stacked_state = self._handle_state(state)
        next_state = self._transform(next_state)
        self.stacked_queue.append(next_state)
        stacked_next_state = self._get_stacked_state()
        self.memory.append(self.MemoryItem(stacked_state, action, reward, stacked_next_state, done))
        
    def sample(self, batch_size):
        indxs = np.random.choice(range(len(self.memory)), batch_size, replace=False)
        return torch.stack([self.memory[i] for i in indxs], 0)

In [46]:
memory = ReplayMemory(device)
game, possible_actions = create_environment()

In [ ]:
game.new_episode()

In [ ]:
state = game.get_state().screen_buffer
for i in range(pretrain_length):
        
    # Random action
    action = random.choice(possible_actions)
    
    # Get the rewards
    reward = game.make_action(action)
    
    # Look if the episode is finished
    done = game.is_episode_finished()
    
    # If we're dead
    if done:
        # We finished the episode
        next_state = np.zeros(state.shape)
        
        # Add experience to memory
        memory.add(state, action, reward, next_state, done)
        
        # Start a new episode
        game.new_episode()
        
        # First we need a state
        state = game.get_state().screen_buffer
        
        # Stack the frames
        state, stacked_frames = stack_frames(stacked_frames, state, True)
        
    else:
        # Get the next state
        next_state = game.get_state().screen_buffer
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
        
        # Add experience to memory
        memory.add((state, action, reward, next_state, done))
        
        # Our state is now the next_state
        state = next_state

## Deep CNN

In [40]:
class DeepQ(nn.Module):
    def __init__(self):
        super(DeepQ, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, 8, 2)
        self.conv2 = nn.Conv2d(32, 64, 4, 2)
        self.conv3 = nn.Conv2d(64, 128, 4, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(8*8*128, 512)
        self.fc2 = nn.Linear(512, 3)
        
    def forward(self, x):
        out_1 = nn.functional.elu(self.bn1(self.conv1(x)))
        out_2 = nn.functional.elu(self.bn2(self.conv2(out_1)))
        out_3 = nn.functional.elu(self.bn3(self.conv3(out_2)))
        out_4 = nn.functional.elu(self.fc1(out_3.view(1, -1)))
        out_5 = self.fc2(out_4)
        return out_5

## Run Deep Q Learning Process

In [ ]:
def initalize_memory(pretrain_length, game):
    game.new_episode()
    for i in range(pretrain_length):
        take_step

In [41]:
game, possible_actions = create_environment()
model = DeepQ().to(device)

In [33]:
model = DeepQ().to(device)

In [34]:
replay = ReplayMemory(device)
replay.push(np.random.rand(120,160))

In [35]:
model(replay.sample(1))

tensor([[-0.1118,  0.3007,  0.1629]])